### Deploy and Interact with LUCE contract
The aim of this notebook is to compile, deploy and interact with all functions of the LUCE smart contract (v3).

In [5]:
name = "fran"

In [6]:
print(name)

fran


### Compile contract from file

In [7]:
# Import libraries
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

In [8]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [9]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [10]:
#compiled_sol

In [11]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [12]:
#contract_interface

In [13]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [14]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [15]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [16]:
# Create contract blueprint
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

In [18]:
# Submit the transaction that deploys the contract
tx_hash = Luce.constructor().transact()

### Obtain Transcation Receipt

In [41]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [42]:
# Obtain address of freshly deployed contract
tx_receipt.contractAddress

'0x1Fe1b20f88e305d4F993Fb67457BcE7863D267f2'

### Interact with contract

In [43]:
# Create python instance of deployed contract
luce1 = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

**publishData**  
```solidity
publishData(string memory _newdescription, string memory _link, uint _licence)
```

In [44]:
# Publish Data
tx_hash = luce1.functions.publishData("example description", "my link", 6).transact()

**setLicence**
```solidity
setLicence(uint newLicence)
```

In [45]:
# Set Licence
_ = luce1.functions.setLicence(28).transact()

From here on I am using `_` to suppress the output of the transaction hash. I do this since transactions are mined instantaneously in the local ganache test setting. On the testnet and mainnet we would store the transaction hash, then wait until a transaction receipt is available and only then we can be sure that the transaction went through.

**getLicence**
```solidity
getLicence() public view returns(uint)
```

In [46]:
# Get Licence
luce1.functions.getLicence().call()

28

**updateData**
```solidity
updateData(string memory updateDescr, string memory _newlink)
```

In [47]:
# Update Data
_ = luce1.functions.updateData("New Description", "New Link").transact()

**getLink**
```solidity
getLink(uint token) public view returns(string memory)
```

In [48]:
luce1.functions.getLink(1).call()

'New Link'

All LUCE functions up to this point are working.

---

We can also query the dataProvider and dataDescription since public variables in smart contracts automatically have a getter method. We simply call them by their name to read the current variable state:

**dataProvider**

In [49]:
luce1.functions.dataProvider().call()

'0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA'

**dataDescription**

In [50]:
luce1.functions.dataDescription().call()

'New Description'

---

The following two functions are not working yet and need debugging:

**addDataRequester**
```solidity
addDataRequester(uint purposeCode, uint licenceType) public returns(uint)
```

In [51]:
luce1.functions.addDataRequester(1,28).transact()

HexBytes('0x5f0dc1271c4ea4175d39614045c3e54122a55a6ea0ecc9e0c20edd062ed97d1a')

This method throws an error at the moment:  
`'message': 'VM Exception while processing transaction: revert`

**renewToken**
```solidity
renewToken(uint compliance) public returns(uint token)
```

In [52]:
luce1.functions.renewToken(2).transact()
# Error: Need to agree on licence first

HexBytes('0xb4114e43fa8e3e9721b915203fab4dadefa1525cc07b78164d761621c221d77d')

`'message': 'VM Exception while processing transaction: revert need to agree on licence first'`